In [1]:
import os
import tweepy

def credentials_dict() -> dict:
    return {
        "consumer_key": os.getenv("twitter_consumer_api_key"),
        "consumer_secret": os.getenv("twitter_consumer_api_secret"),
        "access_token": os.getenv("twitter_access_api_key"),
        "access_token_secret": os.getenv("twitter_access_api_secret")
    }
credentials = credentials_dict()

In [2]:
auth = tweepy.OAuthHandler(credentials["consumer_key"], credentials["consumer_secret"])
auth.set_access_token(credentials["access_token"], credentials["access_token_secret"])
api = tweepy.API(auth)
available_loc = api.available_trends()

In [3]:
available_loc = api.available_trends()

In [4]:
streaming_client = tweepy.StreamingClient(os.environ.get("bearer_token"))
bb = streaming_client.sample()

In [1]:
### v2
from datetime import datetime, timedelta
import os

import tweepy
import pandas as pd
import requests

yesterday_ts = pd.to_datetime(datetime.now()).floor('H') - timedelta(days = 1)

client = tweepy.Client( bearer_token = os.getenv("bearer_token"), 
                        consumer_key = os.getenv("twitter_consumer_api_key"), 
                        consumer_secret = os.getenv("twitter_consumer_api_secret"), 
                        access_token = os.getenv("twitter_access_api_key"), 
                        access_token_secret = os.getenv("twitter_access_api_secret"), 
                        return_type = requests.Response,
                        wait_on_rate_limit = True)

In [46]:
# Define query
query = 'nba'

# get max. 100 tweets - , 'organic_metrics' errors out for me
df = client.search_recent_tweets(query=query, 
                                    tweet_fields=['id', 'created_at', 'author_id', 'geo', 'lang', 'source', 'public_metrics'],
                                     max_results=100)
df = pd.json_normalize(df.json()['data']) 
df.head(5)

,lang,id,source,text,author_id,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,geo.place_id
0,en,1522351152918056960,Twitter for iPhone,"RT @ProvideContext: ""Lebron is lucky he plays ...",292963630,2022-05-05T23:02:53.000Z,90,0,0,0,NaN
1,en,1522351152553250821,Twitter for Android,RT @NBA: Another 1\n\n💎 @djkhaled x #NBA75 htt...,1492060455011065867,2022-05-05T23:02:53.000Z,109,0,0,0,NaN
2,en,1522351150900686848,Twitter for iPhone,RT @RealSkipBayless: The NBA makes me sick. It...,29995876,2022-05-05T23:02:52.000Z,215,0,0,0,NaN
3,en,1522351142893756417,Twitter for Android,RT @Capalot4Mo: NBA Bubble was valid hoops dc ...,2511433761,2022-05-05T23:02:51.000Z,2709,0,0,0,NaN
4,en,1522351141345976330,Twitter for iPhone,@The_DocHolliday @NBA @memgrizz @dillonbrooks2...,361588287,2022-05-05T23:02:50.000Z,0,0,0,0,NaN


In [44]:
query = "nba"

# Granularity is what you want the timeseries count data to be grouped by
# You can request minute, hour, or day granularity
# The default granularity, if not specified is hour

# it groups data by the hour, so yesterday_ts is the yesterday's timestamp rounded down to the nearest hour.
response = client.get_recent_tweets_count(query, granularity="hour", start_time = yesterday_ts)

df = pd.DataFrame(response.json()['data'])